In [15]:
import shutil

source_file_path = '.\..\..\.env'

destination_file_path = '.'

shutil.copy(source_file_path, destination_file_path)

'.\\.env'

In [29]:
import django
import os
import sys
project_path = '../../'  # Adjust this to your actual project path
sys.path.append(project_path)
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()

import re
from asgiref.sync import sync_to_async
from telethon.tl.types import  Message, PeerChannel
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.sync import TelegramClient, events
from tqdm import tqdm
from datetime import datetime, timedelta, timezone
from PostAnalyzer.models import (
    Channel,
    EntryTarget,
    Market,
    Post,
    PostStatus,
    Predict,
    Symbol,
    TakeProfitTarget,
    SettingConfig,
    PositionSide,
    MarginMode,
)
import warnings
from django.forms.models import model_to_dict
from Shared.helpers import returnSearchValue,convertToJsonFile,convertTimestampToDateTime,load_json,print_colored
import pytz
from dotenv import dotenv_values
from Shared.findRiskToReward import findRiskToReward
from Shared.findShortOrderStat import findShortOrderStat
from Shared.findLongOrderStat import findLongOrderStat
from typing import List, Union

config = dotenv_values(".env")

api_id = config["api_id"]
api_hash = config["api_hash"]

username = config["username"]

error_msg = []


allPredicts = []

historic_json_path = "./../historic-json"


# Disable all warnings
warnings.filterwarnings("ignore")

# 1000 $
my_money = 1_000 

tehran_tz = pytz.timezone('Asia/Tehran')


In [17]:
async def findTpProfit(tp_index:int=0, showPrint:bool= False):
    predicts: List[Predict] = await sync_to_async(list)(Predict.objects.all().select_related('post', 'status', 'position', 'symbol'))
    tp_num = tp_index+1
    profit = 0
    length = 0

    for pr in predicts:

        if pr.status.name in ['ERROR', 'CANCELED', 'PENDING']:
            continue 

        # if pr.profit > 0:
        tps = await sync_to_async(
                            lambda: list(TakeProfitTarget.objects.filter(post=pr.post).order_by('index'))
                        )()
        tp_queryset = await sync_to_async(
                            lambda: list(TakeProfitTarget.objects.filter(post=pr.post, active=True).order_by('index'))
                        )()
      
        if len(tp_queryset) == tp_num:
            profit += tps[tp_index].profit
            if showPrint:
                print(f"{pr.symbol.name},\t{pr.status.name}/{pr.status.type},\t\t{pr.date.astimezone(tehran_tz)},\t{round(tps[tp_index].profit,5)},\t{len(tps)}")
            length += 1


    print()
    print(profit, length)
            


await findTpProfit(3, False)        
       

    


993.6681199999998 25


In [18]:
async def findLoss(tp_num:int=1):
    predicts: List[Predict] = await sync_to_async(list)(Predict.objects.all().select_related('post', 'status', 'position', 'symbol'))
    loss = 0

    for pr in predicts:

        if pr.status.name == "FAILED WITH PROFIT":
            if pr.status.type < tp_num:
                loss += pr.profit

        elif pr.status.name == "FAILED":
            loss += pr.profit


    print(loss)


# tp_num = 1 ==> TP0
# tp_num = 2 ==> TP1
# tp_num = 3 ==> TP2
# tp_num = 4 ==> TP3
# tp_num = 5 ==> TP4
await findLoss(5)

-2550.143639999998


In [19]:
def calStoploss(entry:float, leverage:int, isShort:bool, max_percent_stoploss:float):
    return entry*(1+(max_percent_stoploss/(100*leverage*(1 if isShort else -1)))) 

In [20]:
def printDetails(pr:Predict, tp:Union[TakeProfitTarget, None], myMoney, posSize, my_money_open_order, profit, my_money_after, color ):
        if tp:
            print(f"symbol: {pr.symbol.name}, position: {pr.position.name}, status: {pr.status.name}, active/index: {tp.active}/{tp.index}, value: {tp.value}$, {str(tp.profit)}%, date: {pr.date.astimezone(tehran_tz).time()} \n")
        else:
            print(f"symbol: {pr.symbol.name}, position: {pr.position.name}, status: {pr.status.name}, date: {pr.date.astimezone(tehran_tz).time()} \n")

        
        print(f"my current money: {myMoney}")
        print(f"amount of money: {posSize}")

        print(f"my money after open order: {my_money_open_order}")

        print(f"profit of this order: {profit}")

        print(f"my current money after: {my_money_after}")
        print_colored("********************************************************************************************************************", color)

In [21]:
#strategy_1
async def backtest_with_money_strategy_1(my_money:Union[float, int], tp_status_array:List[str], failed_status_array:List[str],close_tp:int=0, showPrint: bool= False, positionSize: Union[float, int]= 100):
    predicts: List[Predict] = await sync_to_async(list)(Predict.objects.all().select_related('post', 'status', 'position', 'symbol'))
    total_loss = 0
    loss_count = 0
    total_profit = 0
    profit_count = 0
    total_pending = 0
    pending_count = 0
    initial_my_money = my_money
    total_opening_orders = 0
    

    is_tp = False
    is_pend= False

    for i, pr in enumerate(predicts):
        is_tp = False
        is_pend = False

        if pr.status.name in failed_status_array:
            
            tp_queryset = await sync_to_async(
                lambda: list(TakeProfitTarget.objects.filter(post=pr.post, active=True).order_by('index'))
            )()
            
            # this "IF" will handle predicts with status of "FAILED WITH PROFIT"
            if len(tp_queryset) > close_tp:
                is_tp = True
            else:
                total_opening_orders += 1
                loss_count += 1

                current_wallet_money = my_money
                my_money -= positionSize
                my_money_open_order = my_money
            
                profit = positionSize*(pr.profit/100) 

                total_loss += profit
                my_money += positionSize + profit
                my_money_after = my_money

                if showPrint:
                    printDetails(pr, None, current_wallet_money, positionSize, my_money_open_order, profit, my_money_after, "red")
       
        if pr.status.name in tp_status_array or is_tp:
            
            tp_queryset = await sync_to_async(
                lambda: list(TakeProfitTarget.objects.filter(post=pr.post).order_by('index'))
            )()

            active_count = 0
            for item in tp_queryset:
                if item.active:
                    active_count += 1


            length = len(tp_queryset)
            is_full = active_count == length
           
            if is_full and length-1 <= close_tp:
                tp = tp_queryset[length-1]

            elif active_count >= close_tp:
                tp = tp_queryset[close_tp]
                
            # this "ELSE" will handle predicts with status of "SUCCESS" which have not reach exact TP or Stoploss
            else:
                is_pend = True
                tp = None

            if tp:
                total_opening_orders += 1
                profit_count += 1
                current_wallet_money = my_money
                my_money -= positionSize
                my_money_open_order = my_money

                profit = positionSize*(tp.profit/100) 

                total_profit += profit
                my_money += positionSize + profit
                my_money_after = my_money

                if showPrint:
                    printDetails(pr, tp, current_wallet_money, positionSize, my_money_open_order, profit, my_money_after, "cyan")

           

        if pr.status.name == "PENDING" or is_pend:
            total_opening_orders += 1
            pending_count += 1

            current_wallet_money = my_money
            my_money -= positionSize
            my_money_open_order = my_money

            total_pending += positionSize
           
            if showPrint:
                printDetails(pr, None, current_wallet_money, positionSize, my_money_open_order, 0, my_money, "grey")
        


    print_colored(f"initial money: {initial_my_money}", "gold")
    print_colored(f"total_opening_orders: {total_opening_orders}", "#1da44f")
    print_colored(f"total_loss: {total_loss}", "pink")
    print_colored(f"total_profit: {total_profit}", "green")
    print_colored(f"gross: {total_profit+total_loss}", "deeppink")
    print_colored(f"total_pending: {total_pending}", "grey")
    print_colored(f"my free money: {my_money}", "orange")
    print_colored(f"my total money: {my_money+total_pending}", "#d16984")
    print_colored(f"profit_count: {profit_count}, loss_count: {loss_count}, pending_count: {pending_count}", "#a1309d")



In [22]:
#strategy_2
async def backtest_with_money_strategy_2(my_money:Union[float, int], close_tp:int=0, showPrint: bool= False, positionSize: Union[float, int]= 100, max_percent_stoploss:float=5):
    predicts: List[Predict] = await sync_to_async(list)(Predict.objects.all().order_by('date').select_related('post', 'status', 'position', 'symbol'))

    total_loss = 0
    loss_count = 0
    total_profit = 0
    profit_count = 0
    total_pending = 0
    pending_count = 0
    initial_my_money = my_money
    total_opening_orders = 0
  

    for i, pr in enumerate(predicts):

        if pr.status.name in ['ERROR', 'CANCELED']:
            continue

        entry_price = await sync_to_async(
            lambda: list(EntryTarget.objects.filter(post=pr.post).order_by('index'))
        )()

        take_profit = await sync_to_async(
            lambda: list(TakeProfitTarget.objects.filter(post=pr.post).order_by('index'))
        )()

        if showPrint:
            print(pr.symbol.name, pr.position.name, pr.date.astimezone(tehran_tz))

        LData = load_json(f"{historic_json_path}/{pr.symbol.name}.json")
        isSHORT = pr.position.name == "SHORT"

        tp_turn = 0
        tp = take_profit[tp_turn].value
        tps = []

        new_entry = entry_price[0].value
        wait_for_entry = True
        entry_reached = []

        stop_loss_reached = None
        start_timestamp = int(pr.date.timestamp() * 1000)
        profit = 0
        # stop_loss = float(pr.stopLoss)
        stop_loss = calStoploss(new_entry, pr.leverage, isSHORT, max_percent_stoploss)
        
        if pr.position.name == "LONG":
            for row in LData:
                if row['time'] < start_timestamp:
                    continue
                
                if wait_for_entry and float(row['low']) <= new_entry:
                    entry_reached.append(row)

                    wait_for_entry = False
                    continue

                if bool(entry_reached) and not wait_for_entry:
                    if float(row['low']) <= stop_loss:
                        profit += round(((stop_loss/new_entry)-1)*100*pr.leverage * 1, 5) 
                        if showPrint:
                            print(f"loss: {((stop_loss/new_entry)-1)*100*pr.leverage}")
                            print(f" new_entry {new_entry} stop_loss {stop_loss} sl profit {profit}")

                        stop_loss_reached = row
                        break

                    if float(row['high']) >= tp: 
                        wait_for_entry = True

                        profit_value = round(((tp/new_entry)-1)*100*pr.leverage * 1, 5)
                        profit += profit_value
                        if showPrint:
                            print(f"tp profit: {profit_value}, total profit {profit}")

                        new_entry = take_profit[tp_turn].value

                        tps.append(row)
                        tp_turn += 1

                        if (tp_turn) == len(take_profit) or close_tp < len(tps):
                            break
                        tp = take_profit[tp_turn].value
                        if tp_turn == 1:
                            stop_loss = entry_price[0].value
                        else:
                            stop_loss = take_profit[tp_turn-2].value
        else:
            for row in LData:
                if row['time'] < start_timestamp:
                    continue

                if wait_for_entry and float(row['high']) >= new_entry:
                    entry_reached.append(row)

                    wait_for_entry = False

                    continue

                if bool(entry_reached) and not wait_for_entry:
                    if float(row['high']) >= stop_loss:
                        profit += round(((stop_loss/new_entry)-1)*100*pr.leverage * -1, 5)  
                        if showPrint:
                            print(f"loss: {-((stop_loss/new_entry)-1)*100*pr.leverage}")
                            print(f"new_entry {new_entry} stop_loss {stop_loss} sl profit {profit}")
                        
                        stop_loss_reached = row
                        break

                    if float(row['low']) <= tp:
                        wait_for_entry = True

                        profit_value = round(((tp/new_entry)-1)*100*pr.leverage * -1, 5)
                        profit += profit_value
                        if showPrint:
                            print(f"tp profit: {profit_value}, total profit {profit}")
                        
                        new_entry = take_profit[tp_turn].value

                        tps.append(row)
                        tp_turn += 1

                        if (tp_turn) == len(take_profit) or close_tp < len(tps):
                            break
                        tp = take_profit[tp_turn].value
                        if tp_turn == 1:
                            stop_loss = entry_price[0].value
                        else:
                            stop_loss = take_profit[tp_turn-2].value


        if profit < 0:
            total_loss += profit
        else:
            total_profit += profit
        stat = {"tps": tps, "entry_reached": entry_reached, "stop_loss_reached": stop_loss_reached}
        if showPrint:
            print(stat)
            print(f"profit: {profit}")
            print("tps")
            [print(convertTimestampToDateTime(x['time'])) for x in tps if x and 'time' in x]

            print("\nentry")
            [print(convertTimestampToDateTime(x['time'])) for x in entry_reached if x and 'time' in x]
                
            if stop_loss_reached: print("\nstop_loss\n", convertTimestampToDateTime(stop_loss_reached['time']))
            
            print_colored("*************************************************************************************************************************************************************************************************************************","#0f0")






    print_colored(f"initial money: {initial_my_money}", "gold")
    print_colored(f"total_opening_orders: {total_opening_orders}", "#1da44f")
    print_colored(f"total_loss: {total_loss}", "pink")
    print_colored(f"total_profit: {total_profit}", "green")
    print_colored(f"gross: {total_profit+total_loss}", "deeppink")
    print_colored(f"total_pending: {total_pending}", "grey")
    print_colored(f"my free money: {my_money}", "orange")
    print_colored(f"my total money: {my_money+total_pending}", "#d16984")
    print_colored(f"profit_count: {profit_count}, loss_count: {loss_count}, pending_count: {pending_count}", "#a1309d")




In [23]:
#strategy_3
async def backtest_with_money_strategy_3(my_money:Union[float, int], close_tp:int=0, showPrint: bool= False, positionSize: Union[float, int]= 100, max_percent_stoploss:float=5, effect_stoploss:bool=True):
    predicts: List[Predict] = await sync_to_async(list)(Predict.objects.all().order_by('date').select_related('post', 'status', 'position', 'symbol'))

    total_loss = 0
    loss_count = 0
    total_profit = 0
    profit_count = 0
    total_pending = 0
    pending_count = 0
    initial_my_money = my_money
    total_opening_orders = 0
    

    for i, pr in enumerate(predicts):

        if pr.status.name in ['ERROR', 'CANCELED']:
            continue

        entry_price = await sync_to_async(
            lambda: list(EntryTarget.objects.filter(post=pr.post).order_by('index'))
        )()

        take_profit = await sync_to_async(
            lambda: list(TakeProfitTarget.objects.filter(post=pr.post).order_by('index'))
        )()

        if showPrint:
            print(pr.symbol.name, pr.position.name, pr.date.astimezone(tehran_tz))

        LData = load_json(f"{historic_json_path}/{pr.symbol.name}.json")
        isSHORT = pr.position.name == "SHORT"

        tp_turn = 0
        tp = take_profit[tp_turn].value
        tps = []

        new_entry = entry_price[0].value
        wait_for_entry = True
        entry_reached = []

        stop_loss_reached = None
        start_timestamp = int(pr.date.timestamp() * 1000)
        profit = 0

        if effect_stoploss:
            stop_loss = calStoploss(new_entry, pr.leverage, isSHORT, max_percent_stoploss)
        else:
            stop_loss = float(pr.stopLoss)
        
        if pr.position.name == "LONG":
            for row in LData:
                if row['time'] < start_timestamp:
                    continue
                
                if wait_for_entry and float(row['low']) <= new_entry:
                    entry_reached.append(row)
                    wait_for_entry = False
                    continue

                if bool(entry_reached) and not wait_for_entry:
                    if float(row['low']) <= stop_loss:
                        profit += round(((stop_loss/new_entry)-1)*100*pr.leverage * 1, 5) 
                        if showPrint:
                            print(f"loss: {((stop_loss/new_entry)-1)*100*pr.leverage}")
                            print(f" new_entry {new_entry} stop_loss {stop_loss} sl profit {profit}")

                        stop_loss_reached = row
                        break

                    if float(row['high']) >= tp: 

                        profit_value = round(((tp/new_entry)-1)*100*pr.leverage * 1, 5)
                        profit += profit_value
                        if showPrint:
                            print(f"tp profit: {profit_value}, total profit {profit}")

                        new_entry = take_profit[tp_turn].value

                        tps.append(row)
                        tp_turn += 1

                        if (tp_turn) == len(take_profit) or close_tp < len(tps):
                            break
                        tp = take_profit[tp_turn].value
                        
        else:
            for row in LData:
                if row['time'] < start_timestamp:
                    continue

                if wait_for_entry and float(row['high']) >= new_entry:
                    entry_reached.append(row)
                    wait_for_entry = False
                    continue

                if bool(entry_reached) and not wait_for_entry:
                    if float(row['high']) >= stop_loss:
                        profit += round(((stop_loss/new_entry)-1)*100*pr.leverage * -1, 5)  
                        if showPrint:
                            print(f"loss: {-((stop_loss/new_entry)-1)*100*pr.leverage}")
                            print(f"new_entry {new_entry} stop_loss {stop_loss} sl profit {profit}")
                        
                        stop_loss_reached = row
                        break

                    if float(row['low']) <= tp:

                        profit_value = round(((tp/new_entry)-1)*100*pr.leverage * -1, 5)
                        profit += profit_value
                        if showPrint:
                            print(f"tp profit: {profit_value}, total profit {profit}")
                        
                        new_entry = take_profit[tp_turn].value

                        tps.append(row)
                        tp_turn += 1

                        if (tp_turn) == len(take_profit) or close_tp < len(tps):
                            break
                        tp = take_profit[tp_turn].value

        if profit < 0:
            total_loss += profit
        else:
            total_profit += profit
        stat = {"tps": tps, "entry_reached": entry_reached, "stop_loss_reached": stop_loss_reached}

        if showPrint:
            print(stat)
            print(f"profit: {profit}")
            print("tps")
            [print(convertTimestampToDateTime(x['time'])) for x in tps if x and 'time' in x]

            print("\nentry")
            [print(convertTimestampToDateTime(x['time'])) for x in entry_reached if x and 'time' in x]
                
            if stop_loss_reached: print("\nstop_loss\n", convertTimestampToDateTime(stop_loss_reached['time']))
            
            print_colored("*************************************************************************************************************************************************************************************************************************","#0f0")




    print_colored(f"initial money: {initial_my_money}", "gold")
    print_colored(f"total_opening_orders: {total_opening_orders}", "#1da44f")
    print_colored(f"total_loss: {total_loss}", "pink")
    print_colored(f"total_profit: {total_profit}", "green")
    print_colored(f"gross: {total_profit+total_loss}", "deeppink")
    print_colored(f"total_pending: {total_pending}", "grey")
    print_colored(f"my free money: {my_money}", "orange")
    print_colored(f"my total money: {my_money+total_pending}", "#d16984")
    print_colored(f"profit_count: {profit_count}, loss_count: {loss_count}, pending_count: {pending_count}", "#a1309d")




In [24]:
#strategy_4
async def backtest_with_money_strategy_4(my_money:Union[float, int], close_tp:int=0, showPrint: bool= False, positionSize: Union[float, int]= 100, max_percent_stoploss:float=5):
    predicts: List[Predict] = await sync_to_async(list)(Predict.objects.all().order_by('date').select_related('post', 'status', 'position', 'symbol'))

    total_loss = 0
    loss_count = 0
    total_profit = 0
    profit_count = 0
    total_pending = 0
    pending_count = 0
    initial_my_money = my_money
    total_opening_orders = 0
  

    for i, pr in enumerate(predicts):

        if pr.status.name in ['ERROR', 'CANCELED']:
            continue

        entry_price = await sync_to_async(
            lambda: list(EntryTarget.objects.filter(post=pr.post).order_by('index'))
        )()

        take_profit = await sync_to_async(
            lambda: list(TakeProfitTarget.objects.filter(post=pr.post).order_by('index'))
        )()

        if showPrint:
            print(pr.symbol.name, pr.position.name, pr.date.astimezone(tehran_tz))

        LData = load_json(f"{historic_json_path}/{pr.symbol.name}.json")
        isSHORT = pr.position.name == "SHORT"

        tp_turn = 0
        tp = take_profit[tp_turn].value
        tps = []

        new_entry = entry_price[0].value
        wait_for_entry = True
        entry_reached = []

        stop_loss_reached = None
        start_timestamp = int(pr.date.timestamp() * 1000)
        profit = 0
        
        stop_loss = calStoploss(new_entry, pr.leverage, isSHORT, max_percent_stoploss)
        
        if pr.position.name == "LONG":
            for row in LData:
                if row['time'] < start_timestamp:
                    continue
                
                if wait_for_entry and float(row['low']) <= new_entry:
                    entry_reached.append(row)
                    wait_for_entry = False
                    continue

                if bool(entry_reached) and not wait_for_entry:
                    if float(row['low']) <= stop_loss:
                        profit += round(((stop_loss/new_entry)-1)*100*pr.leverage * 1, 5) 
                        if showPrint:
                            print(f"loss: {((stop_loss/new_entry)-1)*100*pr.leverage}")
                            print(f" new_entry {new_entry} stop_loss {stop_loss} sl profit {profit}")

                        stop_loss_reached = row
                        break

                    if float(row['high']) >= tp: 

                        tps.append(row)
                        tp_turn += 1

                        if (tp_turn) == len(take_profit) or close_tp < len(tps):
                            profit_value = round(((tp/new_entry)-1)*100*pr.leverage * 1, 5)
                            profit += profit_value
                            if showPrint:
                                print(f"tp profit: {profit_value}, total profit {profit}")

                            break

                        tp = take_profit[tp_turn].value
                        
        else:
            for row in LData:
                if row['time'] < start_timestamp:
                    continue

                if wait_for_entry and float(row['high']) >= new_entry:
                    entry_reached.append(row)
                    wait_for_entry = False
                    continue

                if bool(entry_reached) and not wait_for_entry:
                    if float(row['high']) >= stop_loss:
                        profit += round(((stop_loss/new_entry)-1)*100*pr.leverage * -1, 5)  
                        if showPrint:
                            print(f"loss: {-((stop_loss/new_entry)-1)*100*pr.leverage}")
                            print(f"new_entry {new_entry} stop_loss {stop_loss} sl profit {profit}")
                        
                        stop_loss_reached = row
                        break

                    if float(row['low']) <= tp:

                        tps.append(row)
                        tp_turn += 1

                        if (tp_turn) == len(take_profit) or close_tp < len(tps):
                            profit_value = round(((tp/new_entry)-1)*100*pr.leverage * -1, 5)
                            profit += profit_value
                            if showPrint:
                                print(f"tp profit: {profit_value}, total profit {profit}")

                            break


                        tp = take_profit[tp_turn].value

        if profit < 0:
            total_loss += profit
        else:
            total_profit += profit
        stat = {"tps": tps, "entry_reached": entry_reached, "stop_loss_reached": stop_loss_reached}

        if showPrint:
            print(stat)
            print(f"profit: {profit}")
            print("tps")
            [print(convertTimestampToDateTime(x['time'])) for x in tps if x and 'time' in x]

            print("\nentry")
            [print(convertTimestampToDateTime(x['time'])) for x in entry_reached if x and 'time' in x]
                
            if stop_loss_reached: print("\nstop_loss\n", convertTimestampToDateTime(stop_loss_reached['time']))
            
            print_colored("*************************************************************************************************************************************************************************************************************************","#0f0")




    print_colored(f"initial money: {initial_my_money}", "gold")
    print_colored(f"total_opening_orders: {total_opening_orders}", "#1da44f")
    print_colored(f"total_loss: {total_loss}", "pink")
    print_colored(f"total_profit: {total_profit}", "green")
    print_colored(f"gross: {total_profit+total_loss}", "deeppink")
    print_colored(f"total_pending: {total_pending}", "grey")
    print_colored(f"my free money: {my_money}", "orange")
    print_colored(f"my total money: {my_money+total_pending}", "#d16984")
    print_colored(f"profit_count: {profit_count}, loss_count: {loss_count}, pending_count: {pending_count}", "#a1309d")
    
    return total_profit, total_loss, total_profit+total_loss



In [25]:
#strategy_5
async def backtest_with_money_strategy_5(my_money:Union[float, int], close_tp:int=0, showPrint: bool= False, positionSize: Union[float, int]= 100):
    predicts: List[Predict] = await sync_to_async(list)(Predict.objects.all().order_by('date').select_related('post', 'status', 'position', 'symbol'))

    total_loss = 0
    loss_count = 0
    total_profit = 0
    profit_count = 0
    total_pending = 0
    pending_count = 0
    initial_my_money = my_money
    total_opening_orders = 0
  

    for i, pr in enumerate(predicts):

        if pr.status.name in ['ERROR', 'CANCELED']:
            continue

        entry_price = await sync_to_async(
            lambda: list(EntryTarget.objects.filter(post=pr.post).order_by('index'))
        )()

        take_profit = await sync_to_async(
            lambda: list(TakeProfitTarget.objects.filter(post=pr.post).order_by('index'))
        )()

        if showPrint:
            print(pr.symbol.name, pr.position.name, pr.date.astimezone(tehran_tz))

        LData = load_json(f"{historic_json_path}/{pr.symbol.name}.json")
        isSHORT = pr.position.name == "SHORT"

        tp_turn = 0
        tp = take_profit[tp_turn].value
        tps = []

        new_entry = entry_price[0].value
        wait_for_entry = True
        entry_reached = []

        stop_loss_reached = None
        start_timestamp = int(pr.date.timestamp() * 1000)
        profit = 0
        # stop_loss = float(pr.stopLoss)

        index = close_tp if len(take_profit)-1 >= close_tp else len(take_profit)-1

        percent = round(((take_profit[index].value-new_entry)/new_entry)*pr.leverage*100*(-1 if isSHORT else 1), 2)
        stop_loss = calStoploss(new_entry, pr.leverage, isSHORT, percent)
        if showPrint:
            print(f"percent: {percent}")
        
        if pr.position.name == "LONG":
            for row in LData:
                if row['time'] < start_timestamp:
                    continue
                
                if wait_for_entry and float(row['low']) <= new_entry:
                    entry_reached.append(row)

                    wait_for_entry = False
                    continue

                if bool(entry_reached) and not wait_for_entry:
                    if float(row['low']) <= stop_loss:
                        # profit += round(((stop_loss/new_entry)-1)*100*pr.leverage * 1, 5) 
                        profit = round(((stop_loss/new_entry)-1)*100*pr.leverage * 1, 5) 
                        if showPrint:
                            print(f" new_entry {new_entry} stop_loss {stop_loss} sl profit {profit}")

                        stop_loss_reached = row
                        break

                    if float(row['high']) >= tp: 
                        # wait_for_entry = True

                        profit_value = round(((tp/new_entry)-1)*100*pr.leverage * 1, 5)
                        profit = profit_value
                        # profit += profit_value
                        if showPrint:
                            print(f"tp profit: {profit_value}, total profit {profit}")

                        # new_entry = take_profit[tp_turn].value

                        tps.append(row)
                        tp_turn += 1

                        if (tp_turn) == len(take_profit) or close_tp < len(tps):
                            break
                        tp = take_profit[tp_turn].value
                        # if tp_turn == 1:
                        #     stop_loss = entry_price[0].value
                        # else:
                        #     stop_loss = take_profit[tp_turn-2].value
        else:
            for row in LData:
                if row['time'] < start_timestamp:
                    continue

                if wait_for_entry and float(row['high']) >= new_entry:
                    entry_reached.append(row)

                    wait_for_entry = False

                    continue

                if bool(entry_reached) and not wait_for_entry:
                    if float(row['high']) >= stop_loss:
                        profit = round(((stop_loss/new_entry)-1)*100*pr.leverage * -1, 5)  
                        # profit += round(((stop_loss/new_entry)-1)*100*pr.leverage * -1, 5)  
                        if showPrint:
                            print(f"new_entry {new_entry} stop_loss {stop_loss} sl profit {profit}")
                        
                        stop_loss_reached = row
                        break

                    if float(row['low']) <= tp:
                        # wait_for_entry = True

                        profit_value = round(((tp/new_entry)-1)*100*pr.leverage * -1, 5)
                        profit = profit_value
                        # profit += profit_value
                        if showPrint:
                            print(f"tp profit: {profit_value}, total profit {profit}")
                        
                        # new_entry = take_profit[tp_turn].value

                        tps.append(row)
                        tp_turn += 1

                        if (tp_turn) == len(take_profit) or close_tp < len(tps):
                            break
                        tp = take_profit[tp_turn].value
                        # if tp_turn == 1:
                        #     stop_loss = entry_price[0].value
                        # else:
                        #     stop_loss = take_profit[tp_turn-2].value


        if profit < 0:
            total_loss += profit
        else:
            total_profit += profit
        
        stat = {"tps": tps, "entry_reached": entry_reached, "stop_loss_reached": stop_loss_reached}
        
        if showPrint:
            print(stat)
            print(f"profit: {profit}")
            print("tps")
            [print(convertTimestampToDateTime(x['time'])) for x in tps if x and 'time' in x]

            print("\nentry")
            [print(convertTimestampToDateTime(x['time'])) for x in entry_reached if x and 'time' in x]
                
            if stop_loss_reached: print("\nstop_loss\n", convertTimestampToDateTime(stop_loss_reached['time']))
            
            print_colored("*************************************************************************************************************************************************************************************************************************","#0f0")



    print_colored(f"initial money: {initial_my_money}", "gold")
    print_colored(f"total_opening_orders: {total_opening_orders}", "#1da44f")
    print_colored(f"total_loss: {total_loss}", "pink")
    print_colored(f"total_profit: {total_profit}", "green")
    print_colored(f"gross: {total_profit+total_loss}", "deeppink")
    print_colored(f"total_pending: {total_pending}", "grey")
    print_colored(f"my free money: {my_money}", "orange")
    print_colored(f"my total money: {my_money+total_pending}", "#d16984")
    print_colored(f"profit_count: {profit_count}, loss_count: {loss_count}, pending_count: {pending_count}", "#a1309d")




In [26]:
#strategy_6
async def backtest_with_money_strategy_6(my_money:Union[float, int], trailing_stop_percent:float, close_tp:int=0, showPrint: bool= False, positionSize: Union[float, int]= 100):
    predicts: List[Predict] = await sync_to_async(list)(Predict.objects.all().order_by('date').select_related('post', 'status', 'position', 'symbol'))

    total_loss = 0
    loss_count = 0
    total_profit = 0
    profit_count = 0
    total_pending = 0
    pending_count = 0
    initial_my_money = my_money
    total_opening_orders = 0
  

    for i, pr in enumerate(predicts):

        # if pr.status.name in ['ERROR', 'CANCELED']:
        if pr.status.name in ['ERROR', 'CANCELED'] or pr.leverage > 6:
            continue

        entry_price: List[EntryTarget]= await sync_to_async(
            lambda: list(EntryTarget.objects.filter(post=pr.post).order_by('index'))
        )()

        take_profit: List[TakeProfitTarget] = await sync_to_async(
            lambda: list(TakeProfitTarget.objects.filter(post=pr.post).order_by('index'))
        )()

        if showPrint:
            print(pr.symbol.name, pr.position.name, pr.date.astimezone(tehran_tz))

        LData = load_json(f"{historic_json_path}/{pr.symbol.name}.json")
        isSHORT = pr.position.name == "SHORT"

        tp_turn = 0
        tp = take_profit[tp_turn].value
        tps = []

        new_entry = entry_price[0].value
        wait_for_entry = True
        entry_reached = []

        stop_loss_reached = None
        start_timestamp = int(pr.date.timestamp() * 1000)
        profit = 0
        # stop_loss = float(pr.stopLoss)
        # leverage = 4
        leverage = pr.leverage
        stop_loss_trailing_value = new_entry
        stop_loss = calStoploss(stop_loss_trailing_value, leverage, isSHORT, trailing_stop_percent)
        if showPrint:
            print(f"trailing_stop_percent: {trailing_stop_percent}, stop_loss: {stop_loss}")
        
        if pr.position.name == "LONG":
            for row in LData:
                if row['time'] < start_timestamp:
                    continue
                
                if wait_for_entry and float(row['low']) <= new_entry:
                    entry_reached.append(row)

                    wait_for_entry = False
                    continue

                if bool(entry_reached) and not wait_for_entry:
                    if float(row['low']) <= stop_loss:
                        profit = round(((stop_loss/new_entry)-1)*100*leverage * 1, 5) 
                        # profit += round(((stop_loss/new_entry)-1)*100*leverage * 1, 5) 
                        if showPrint:
                            print(f" new_entry {new_entry} stop_loss {stop_loss} sl profit {profit}")

                        stop_loss_reached = row
                        break

                    if float(row['high']) > stop_loss_trailing_value:
                        stop_loss_trailing_value = float(row['high'])
                        stop_loss = calStoploss(stop_loss_trailing_value, leverage, isSHORT, trailing_stop_percent)


                    # if float(row['high']) >= tp: 
                    #     profit_value = round(((tp/new_entry)-1)*100*leverage * 1, 5)
                    #     profit = profit_value
                    #     # profit += profit_value
                    #     if showPrint:
                    #         print(f"tp profit: {profit_value}, total profit {profit}")

                    #     # new_entry = take_profit[tp_turn].value

                    #     tps.append(row)
                    #     tp_turn += 1

                    #     if tp_turn == len(take_profit) or close_tp < len(tps):
                    #         break
                    #     tp = take_profit[tp_turn].value
                        
        else:
            for row in LData:
                if row['time'] < start_timestamp:
                    continue

                if wait_for_entry and float(row['high']) >= new_entry:
                    entry_reached.append(row)

                    wait_for_entry = False

                    continue

                if bool(entry_reached) and not wait_for_entry:
                    if float(row['high']) >= stop_loss:
                        profit = round(((stop_loss/new_entry)-1)*100*leverage * -1, 5)  
                        # profit += round(((stop_loss/new_entry)-1)*100*leverage * -1, 5)  
                        if showPrint:
                            print(f"new_entry {new_entry} stop_loss {stop_loss} sl profit {profit}")
                        
                        stop_loss_reached = row
                        break

                    if float(row['low']) < stop_loss_trailing_value:
                        stop_loss_trailing_value = float(row['low'])
                        stop_loss = calStoploss(stop_loss_trailing_value, leverage, isSHORT, trailing_stop_percent)


                    # if float(row['low']) <= tp:
                    #     profit_value = round(((tp/new_entry)-1)*100*leverage * -1, 5)
                    #     profit = profit_value
                    #     # profit += profit_value
                    #     if showPrint:
                    #         print(f"tp profit: {profit_value}, total profit {profit}")
                        
                    #     # new_entry = take_profit[tp_turn].value

                    #     tps.append(row)
                    #     tp_turn += 1

                    #     if tp_turn == len(take_profit) or close_tp < len(tps):
                    #         break
                    #     tp = take_profit[tp_turn].value
                    #     # if tp_turn == 1:
                    #     #     stop_loss = entry_price[0].value
                    #     # else:
                    #     #     stop_loss = take_profit[tp_turn-2].value


        if profit < 0:
            total_loss += profit
        else:
            total_profit += profit
       
        stat = {"tps": tps, "entry_reached": entry_reached, "stop_loss_reached": stop_loss_reached}
       
        if showPrint:
            print(stat)
            print(f"profit: {profit}")
            print("tps")
            [print(convertTimestampToDateTime(x['time'])) for x in tps if x and 'time' in x]

            print("\nentry")
            [print(convertTimestampToDateTime(x['time'])) for x in entry_reached if x and 'time' in x]
                
            if stop_loss_reached: print("\nstop_loss\n", convertTimestampToDateTime(stop_loss_reached['time']))
            
            print_colored("*************************************************************************************************************************************************************************************************************************","#0f0")


    print_colored(f"initial money: {initial_my_money}", "gold")
    print_colored(f"total_opening_orders: {total_opening_orders}", "#1da44f")
    print_colored(f"total_loss: {total_loss}", "pink")
    print_colored(f"total_profit: {total_profit}", "green")
    print_colored(f"gross: {total_profit+total_loss}", "deeppink")
    print_colored(f"total_pending: {total_pending}", "grey")
    print_colored(f"my free money: {my_money}", "orange")
    print_colored(f"my total money: {my_money+total_pending}", "#d16984")
    print_colored(f"profit_count: {profit_count}, loss_count: {loss_count}, pending_count: {pending_count}", "#a1309d")




In [27]:
#strategy_7
async def backtest_with_money_strategy_7(my_money:Union[float, int], close_tp:int=0, showPrint: bool= False, positionSize: Union[float, int]= 100, max_percent_stoploss:float=5):
    predicts: List[Predict] = await sync_to_async(list)(Predict.objects.all().order_by('date').select_related('post', 'status', 'position', 'symbol'))

    total_loss = 0
    loss_count = 0
    total_profit = 0
    profit_count = 0
    total_pending = 0
    pending_count = 0
    initial_my_money = my_money
    total_opening_orders = 0
  

    for i, pr in enumerate(predicts):

        if pr.status.name in ['ERROR', 'CANCELED'] or pr.leverage >6:
            continue

        entry_price = await sync_to_async(
            lambda: list(EntryTarget.objects.filter(post=pr.post).order_by('index'))
        )()

        take_profit = await sync_to_async(
            lambda: list(TakeProfitTarget.objects.filter(post=pr.post).order_by('index'))
        )()

        if showPrint:
            print(pr.symbol.name, pr.position.name, pr.date.astimezone(tehran_tz))

        LData = load_json(f"{historic_json_path}/{pr.symbol.name}.json")
        isSHORT = pr.position.name == "SHORT"

        tp_turn = 0
        tp = take_profit[tp_turn].value
        tps = []

        new_entry = entry_price[0].value
        wait_for_entry = True
        entry_reached = []

        stop_loss_reached = None
        start_timestamp = int(pr.date.timestamp() * 1000)
        profit = 0
        # stop_loss = float(pr.stopLoss)
        stop_loss = calStoploss(new_entry, pr.leverage, isSHORT, max_percent_stoploss)
        
        if pr.position.name == "LONG":
            for row in LData:
                if row['time'] < start_timestamp:
                    continue
                
                if wait_for_entry and float(row['low']) <= new_entry:
                    entry_reached.append(row)

                    wait_for_entry = False
                    continue

                if bool(entry_reached) and not wait_for_entry:
                    if float(row['low']) <= stop_loss:
                        profit += round(((stop_loss/new_entry)-1)*100*pr.leverage * 1, 5) 
                        if showPrint:
                            print(f"loss: {((stop_loss/new_entry)-1)*100*pr.leverage}")
                            print(f" new_entry {new_entry} stop_loss {stop_loss} sl profit {profit}")

                        stop_loss_reached = row
                        break

                    if float(row['high']) >= tp: 
                        wait_for_entry = True

                        profit_value = round(((tp/new_entry)-1)*100*pr.leverage * 1, 5)
                        profit += profit_value
                        if showPrint:
                            print(f"tp profit: {profit_value}, total profit {profit}")

                        new_entry = take_profit[tp_turn].value

                        tps.append(row)
                        tp_turn += 1

                        if (tp_turn) == len(take_profit) or close_tp < len(tps):
                            break
                        tp = take_profit[tp_turn].value
                        if tp_turn == 1:
                            stop_loss = entry_price[0].value
                        else:
                            stop_loss = take_profit[tp_turn-2].value
        else:
            for row in LData:
                if row['time'] < start_timestamp:
                    continue

                if wait_for_entry and float(row['high']) >= new_entry:
                    entry_reached.append(row)

                    wait_for_entry = False

                    continue

                if bool(entry_reached) and not wait_for_entry:
                    if float(row['high']) >= stop_loss:
                        profit += round(((stop_loss/new_entry)-1)*100*pr.leverage * -1, 5)  
                        if showPrint:
                            print(f"loss: {-((stop_loss/new_entry)-1)*100*pr.leverage}")
                            print(f"new_entry {new_entry} stop_loss {stop_loss} sl profit {profit}")
                        
                        stop_loss_reached = row
                        break

                    if float(row['low']) <= tp:
                        wait_for_entry = True

                        profit_value = round(((tp/new_entry)-1)*100*pr.leverage * -1, 5)
                        profit += profit_value
                        if showPrint:
                            print(f"tp profit: {profit_value}, total profit {profit}")
                        
                        new_entry = take_profit[tp_turn].value

                        tps.append(row)
                        tp_turn += 1

                        if (tp_turn) == len(take_profit) or close_tp < len(tps):
                            break
                        tp = take_profit[tp_turn].value
                        if tp_turn == 1:
                            stop_loss = entry_price[0].value
                        else:
                            stop_loss = take_profit[tp_turn-2].value


        if profit < 0:
            total_loss += profit
        else:
            total_profit += profit
        stat = {"tps": tps, "entry_reached": entry_reached, "stop_loss_reached": stop_loss_reached}
        if showPrint:
            print(stat)
            print(f"profit: {profit}")
            print("tps")
            [print(convertTimestampToDateTime(x['time'])) for x in tps if x and 'time' in x]

            print("\nentry")
            [print(convertTimestampToDateTime(x['time'])) for x in entry_reached if x and 'time' in x]
                
            if stop_loss_reached: print("\nstop_loss\n", convertTimestampToDateTime(stop_loss_reached['time']))
            
            print_colored("*************************************************************************************************************************************************************************************************************************","#0f0")






    print_colored(f"initial money: {initial_my_money}", "gold")
    print_colored(f"total_opening_orders: {total_opening_orders}", "#1da44f")
    print_colored(f"total_loss: {total_loss}", "pink")
    print_colored(f"total_profit: {total_profit}", "green")
    print_colored(f"gross: {total_profit+total_loss}", "deeppink")
    print_colored(f"total_pending: {total_pending}", "grey")
    print_colored(f"my free money: {my_money}", "orange")
    print_colored(f"my total money: {my_money+total_pending}", "#d16984")
    print_colored(f"profit_count: {profit_count}, loss_count: {loss_count}, pending_count: {pending_count}", "#a1309d")




In [28]:
my_money = 1_000 

# have_tp = ["SUCCESS", "FULLTARGET", "FAILED WITH PROFIT"]
# failed = ["FAILED"]

have_tp_type_2 = ["SUCCESS", "FULLTARGET"]
failed_type_2 = ["FAILED", "FAILED WITH PROFIT"]

# await backtest_with_money_strategy_1(my_money, have_tp_type_2, failed_type_2, 4, False, 100)
# print_colored("*************************************************************************************************************************************************************************************************************************","#0f5666")
# await backtest_with_money_strategy_2(my_money, 1, True, 100, 1)
# for x in range(1, 11):
#     print_colored(f"**********************************************************************************************************************{x}***************************************************************************************************","#0f0")

#     for i in range(0, 5):
#         print_colored(f"****************************************************************************************************************{i}*********************************************************************************************************","##f0f")
#         await backtest_with_money_strategy_2(my_money, i, False, 100, x)
#         # await backtest_with_money_strategy_2(my_money, i, False, 100, x-0.5)

# for x in range(1, 2):
#     print_colored(f"**********************************************************************************************************************{x}***************************************************************************************************","#0f0")

#     for i in range(0, 5):
#         print_colored(f"****************************************************************************************************************{i}*********************************************************************************************************","##f0f")
#         print_colored(f"****************************************************************************************************************true*********************************************************************************************************","#8251fc")
#         await backtest_with_money_strategy_3(my_money, i, False, 100, x-0.5, True)
#         print_colored(f"****************************************************************************************************************False*********************************************************************************************************","#d00f41")
#         await backtest_with_money_strategy_3(my_money, i, False, 100, x-0.5, False)


# for x in range(1, 11):
#         print_colored(f"**********************************************************************************************************************{x}***************************************************************************************************","#0f0")
#         for i in range(0, 5):
#                 print_colored(f"****************************************************************************************************************{i}*********************************************************************************************************","##f0f")
#                 await backtest_with_money_strategy_4(my_money, i, False, 100, x*0.5)


# await backtest_with_money_strategy_1(my_money, have_tp_type_2, failed_type_2, 0, False, 100)
# first = 0.5


# y_profit = []
# y_loss = []
# y_gross = []

# # for i in range(0, 5):
# #     # print_colored(f"*******************************************{i}****************************************************************************************************************************************************************************","#f0f")

# for num in range(0, 41):
#     # print_colored(f"*******************************************{first+(0.05*num)}****************************************************************************************************************************************************************************","#0f0")
#     profit , loss, gross = await backtest_with_money_strategy_4(my_money, 3, False, 100, first+(0.05*num))
#     y_profit.append(round(profit,2))
#     y_loss.append(round(loss,2))
#     y_gross.append(round(gross,2))


# print(y_profit)
# print()
# print(y_loss)
# print()
# print(y_gross)

# await backtest_with_money_strategy_5(my_money, 0, False, 100)
# await backtest_with_money_strategy_5(my_money, 1, False, 100)
# await backtest_with_money_strategy_5(my_money, 2, False, 100)
# await backtest_with_money_strategy_5(my_money, 3, False, 100)
# await backtest_with_money_strategy_5(my_money, 4, False, 100)



# await backtest_with_money_strategy_6(my_money, 1.5, 0, False, 100)
# await  backtest_with_money_strategy_7(my_money, 0, False, 100, 1)
await  backtest_with_money_strategy_7(my_money, 1, False, 100, 1.25)
await  backtest_with_money_strategy_7(my_money, 2, False, 100, 1.25)
await  backtest_with_money_strategy_7(my_money, 3, False, 100, 1.25)
await  backtest_with_money_strategy_7(my_money, 4, False, 100, 1.25)
